# Ignite cheat sheet: Engine, Event, and Hander

This cheat sheet is created from the content of [Introduction to PyTorch-Ignite](https://pytorch-ignite.ai/blog/introduction/)

## Common PyTorch code
First, we define our model, training and validation datasets, optimizer and loss function as normal `Pytorch` program.

In [ ]:
# transform to normalize the data
transform = Compose([ToTensor(), Normalize((0.1307,), (0.3081,))])

# Download and load the training data
trainset = MNIST("data", download=True, train=True, transform=transform)
train_loader = DataLoader(trainset, batch_size=128, shuffle=True)
# Download and load the test data
validationset = MNIST("data", train=False, transform=transform)
val_loader = DataLoader(validationset, batch_size=256, shuffle=False)

# Define a class of CNN model (as you want)
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        ...Define cnn layers
    def forward(self, x):
        ... Do the forward step

device = "cuda"

# Define a model on move it on CUDA device
model = Net().to(device)

# Define a loss
criterion = nn.NLLLoss()

# Define an optimizer
optimizer = SGD(model.parameters(), lr=0.01, momentum=0.8)

## Engine
The [Engine](https://pytorch.org/ignite/engine.html) is responsible for running an arbitrary function - typically a training or evaluation function - and emitting events along the way.

In [ ]:
from ignite.engine import Engine

def train_step(engine, batch):  # train_step function must accept engine and batch arguments. 
    x, y = batch, x = x.to(device), y = y.to(device)
    model.train(), y_pred = model(x), loss = criterion(y_pred, y)
    optimizer.zero_grad(), loss.backward(), optimizer.step()
    return loss

# Define a trainer engine
trainer = Engine(train_step)

In [ ]:
def validation_step(engine, batch):
    model.eval()
    with torch.no_grad():
        x, y = batch[0], batch[1], x = x.to("cuda"), y = y.to("cuda")
        y_pred = model(x)
        return y_pred, y
evaluator = Engine(validation_step)

- Similarly, model evaluation, `evaluator` can be done with an engine that runs a single time over the validation dataset and computes metrics
- `trainer` and `evaluator` and any other functions can be considered `process functions`, that can return `loss`, `y`, `y_pred`, or anything that you want. 

## Events and Handers

- Event system is introduced to facilitate the interaction on each step of the run. 
- **`Engine` allows to add `handlers` on various `Events` that are triggered during the run**. 
- When an event is triggered, attached `handlers` (named functions, lambdas, class method, or any other functions) are executed. 

In [ ]:
# Schema for when built-in events are triggered by default:
fire_event(Events.STARTED)
while epoch < max_epochs:
    fire_event(Events.EPOCH_STARTED)
    # run once on data
    for batch in data:
        fire_event(Events.ITERATION_STARTED)
        output = process_function(batch)  #Note: trainer and evaluator are process functions
        fire_event(Events.ITERATION_COMPLETED)
    fire_event(Events.EPOCH_COMPLETED)
fire_event(Events.COMPLETED)

- **Each engine (i.e. `trainer` and `evaluator`, or other engine you define) has its own event system which allows to define its own engine’s process logic**.

```python
engine_name.add_event_handler(Events.eventname, function_name)
```

In [ ]:
#Using Events and handlers, it is possible to completely customize the engine’s runs in a very intuitive way:

# Show a message when the training begins
@trainer.on(Events.STARTED)
def start_message():
    print("Start training!")

# Handler can be what you want, named functions, lambdas, class method, or any other functions, here a lambda !
trainer.add_event_handler(
    Events.COMPLETED,
    lambda _: print("Training completed!")
)

# Run evaluator on val_loader every trainer's epoch completed
# the run_validation function is attached to the trainer and will be triggered at each completed epoch 
# to launch model’s validation with evaluator. 
@trainer.on(Events.EPOCH_COMPLETED)
def run_validation():
    evaluator.run(val_loader)

Click to know about decorator [trainer.on](https://pytorch.org/ignite/generated/ignite.engine.engine.Engine.html#ignite.engine.engine.Engine.on) and [trainer.add_event_handler](https://pytorch.org/ignite/generated/ignite.engine.engine.Engine.html#ignite.engine.engine.Engine.add_event_handler)

## Reference
- [https://pytorch-ignite.ai/tutorials/beginner/01-getting-started/](https://pytorch-ignite.ai/tutorials/beginner/01-getting-started/) 
- [https://pytorch-ignite.ai/blog/introduction/](https://pytorch-ignite.ai/blog/introduction/)